In [1]:
import csv
import mapreduce as mr

In [3]:
filename = "../data/citibike.csv"
with open (filename,"r") as fi:
    reader = csv.DictReader(fi)
    for ride in reader:
        print ride
        break

{'end_station_id': '423', 'gender': '2', 'bikeid': '17131', 'start_station_latitude': '40.75044999', 'end_station_name': 'W 54 St & 9 Ave', 'cartodb_id': '1', 'start_station_name': '8 Ave & W 31 St', 'start_station_id': '521', 'start_station_longitude': '-73.99481051', 'usertype': 'Subscriber', 'stoptime': '2015-02-01 00:14:00+00', 'end_station_longitude': '-73.98690506', 'starttime': '2015-02-01 00:00:00+00', 'end_station_latitude': '40.76584941', 'tripduration': '801', 'the_geom': '', 'birth_year': '1978'}


In [85]:
filename = "../data/citibike_no_headers.csv"
with open (filename,"r") as fi:
    reader = csv.reader(fi)
    for ride in reader:
        print ride
        break

['1', '', '801', '2015-02-01 00:00:00+00', '2015-02-01 00:14:00+00', '521', '8 Ave & W 31 St', '40.75044999', '-73.99481051', '423', 'W 54 St & 9 Ave', '40.76584941', '-73.98690506', '17131', 'Subscriber', '1978', '2']


In [118]:
filename = "../data/citibike_no_headers.csv"
def mapper(reader):
    mins = {}
    for row in reader:
        items = row.strip('\n').split(',')
        trip_min = int(items[2])/60
        mins[trip_min] = mins.get(trip_min,0)+1
    yield (0,mins)

with open (filename,"r") as fi:
    lst_mins = list(mr.runPartition(fi,mapper, chunk_size=10000))
print lst_mins

[(0, {1: 172, 2: 336, 3: 572, 4: 799, 5: 867, 6: 854, 7: 826, 8: 706, 9: 671, 10: 574, 11: 512, 12: 477, 13: 365, 14: 317, 15: 252, 16: 217, 17: 170, 18: 150, 19: 126, 20: 115, 21: 97, 22: 90, 23: 82, 24: 86, 25: 54, 26: 53, 27: 33, 28: 59, 29: 47, 30: 26, 31: 23, 32: 31, 33: 19, 34: 22, 35: 15, 36: 23, 37: 12, 38: 8, 39: 10, 40: 6, 41: 8, 42: 10, 43: 4, 44: 4, 45: 2, 46: 5, 47: 2, 48: 1, 49: 7, 50: 2, 51: 4, 52: 1, 53: 4, 54: 2, 55: 3, 56: 1, 57: 2, 58: 2, 60: 1, 61: 1, 62: 3, 63: 1, 66: 1, 523: 1, 69: 2, 70: 2, 72: 2, 585: 1, 74: 1, 587: 1, 76: 1, 84: 1, 87: 3, 88: 1, 92: 1, 94: 1, 95: 1, 96: 1, 98: 2, 99: 1, 106: 1, 627: 1, 116: 1, 118: 1, 120: 1, 127: 1, 128: 1, 131: 1, 138: 1, 139: 2, 159: 1, 165: 1, 166: 1, 517: 1, 185: 1, 188: 1, 713: 1, 716: 1, 224: 1, 519: 1, 248: 1, 254: 1, 520: 1, 294: 1, 295: 1, 322: 1, 353: 1, 363: 1, 364: 1, 232: 1, 441: 1}), (0, {1: 81, 2: 198, 3: 415, 4: 680, 5: 747, 6: 877, 7: 837, 8: 845, 9: 733, 10: 645, 11: 570, 12: 477, 13: 394, 14: 317, 15: 266, 1

In [117]:
len(lst_mins)

5

In [148]:
#2:

def mapper(fi):
    mins = {}#initializing an empty dictionary
    for row in fi: #type(row)=str
        items = row.strip('\n').split(',')#type(items)=list
        trip_min = int(items[2])/60 
        mins[trip_min] = mins.get(trip_min,0)+1 
        # mins[trip_min] is syntax for assigning value to key in dictionary corresponding to trip_min
        #this calls up the existing value corresponding to key of trip_min, and adds 1 to it. if there is no value, it assigs 0, and adds one to it
    yield (0,mins) #after all 10,000 rows are done, yields a tupule containing 0 and the dictionary
    
    #(because mr expects a key and a count, and will shuffle all the stuff corresponding to the same keys (which is all 5 chunks' dictionaries) together into a list)

#4
def addBins((bins,tot_count),one_chunk_of_min_dicts):
    for (m,c) in one_chunk_of_min_dicts.iteritems():
        bins[m] = bins.get(m,0)+c
        tot_count +=c
    return (bins, tot_count)

#3
def reducer((now_useless_zero, min_dicts_as_list)):
    #Remember: reduce basically does: addBins( ({},0) , mins_as_list[0] )
    #                      and then : addBins( returned(bins, tot_count) , mins_as_list[1] ) and so on for the next dictionaries
    
    (bins, total_count) = reduce(addBins, min_dicts_as_list, ({}, 0))
    #the above 'bins' now contains a single dictionary, containing all the trip_durs and their frequency, as well as total count (=46200)
    #We will use this number, below, to calculate the median trip_dur(in mins)
    count = 0
    for (m,c) in bins.iteritems():
        count+=c
        if count>total_count/2:
            return m
    return 0

#1:
filename = "../data/citibike_no_headers.csv"
with open (filename,"r") as fi: #type(fi)='file'
    med_tripdur = list(mr.runPartition(fi,mapper, reducer, chunk_size=10000))#46,200 rows = 5 chunks
print "The Median trip duration is {} minutes" .format(med_tripdur[0])

The Median trip duration is 8 minutes
